In [3]:
import pandas as pd
import spacy
from transformers import BertTokenizer, BertForMaskedLM
from util import preprocess_data

data = pd.read_csv('../data/recruitment_data_modified_python.csv',
                   encoding='utf-8',
                   sep=',',
                   on_bad_lines='warn',
                   quotechar='"',
                   doublequote=True,
                   names=['URL', 'Voivodeship', 'Scrap_time', 'Name', 'Price', 'Brand', 'Condition', 'Offer_from', 'Type', 'Description', 'Added_at', 'Views', 'User_since'],
                   skiprows=1)

preprocessed_data = preprocess_data(data)

Skipping line 346: expected 13 fields, saw 14
Skipping line 367: expected 13 fields, saw 15
Skipping line 466: expected 13 fields, saw 19
Skipping line 467: expected 13 fields, saw 19
Skipping line 494: expected 13 fields, saw 15
Skipping line 615: expected 13 fields, saw 15
Skipping line 616: expected 13 fields, saw 15
Skipping line 617: expected 13 fields, saw 15
Skipping line 618: expected 13 fields, saw 15
Skipping line 619: expected 13 fields, saw 15
Skipping line 620: expected 13 fields, saw 15
Skipping line 712: expected 13 fields, saw 14
Skipping line 713: expected 13 fields, saw 14
Skipping line 739: expected 13 fields, saw 14
Skipping line 747: expected 13 fields, saw 16
Skipping line 867: expected 13 fields, saw 15
Skipping line 956: expected 13 fields, saw 14
Skipping line 1028: expected 13 fields, saw 14
Skipping line 1234: expected 13 fields, saw 15
Skipping line 1282: expected 13 fields, saw 17
Skipping line 1326: expected 13 fields, saw 14
Skipping line 1327: expected 1

In [4]:
preprocessed_data

,Price,Date,Concatenated_description
0,2799.0,2021-02-22,iphone 11 64 jak nowy 95% gwarancja wyświetlac...
1,2700.0,2021-02-21,"iphone 11 64 gb czarny, idealny z gwarancją. w..."
2,2899.0,2021-02-21,jak nowy apple iphone 11 256gbgb white gwaranc...
3,2500.0,2021-02-21,apple iphone 11 biały 64gb - jak nowy gwarancj...
4,2150.0,2021-02-21,"iphone 11 64 gb + gwarancja witam, mam na sprz..."
...,...,...,...
2667,2299.0,2021-02-21,iphone 11 black 64gb sprzedam iphone 11 64 gb ...
2668,1900.0,2021-02-21,i phone 11 64 gb cena tylko dzis cena tylko dz...
2669,2800.0,2021-02-21,"iphone 11 128 gb gwarancja , 100% bateria spr..."
2670,1650.0,2021-02-20,iphone 11 white 64gb na sprzedaż posiadam ipho...


In [15]:
nlp = spacy.load('pl_core_news_lg')

In [31]:
preprocessed_data['Concatenated_description'][0]

'iphone 11 64 jak nowy 95% gwarancja wyświetlacz jak nowy . kondycja baterii 95%. kupiony w media markt . posiadam faktury . dodatkowa gwarancja na zbity wyświetlacz wartość 600zł . dodatkowo szkło hartowane 5d oraz pokrowiec spigen wartość 80zl. nie sprzedaje za granice !! . polecam'

In [35]:
doc = nlp.tokenizer(preprocessed_data['Concatenated_description'][0])
print([w.text for w in doc])

['iphone', '11', '64', 'jak', 'nowy', '95', '%', 'gwarancja', 'wyświetlacz', 'jak', 'nowy', '.', 'kondycja', 'baterii', '95', '%', '.', 'kupiony', 'w', 'media', 'markt', '.', 'posiadam', 'faktury', '.', 'dodatkowa', 'gwarancja', 'na', 'zbity', 'wyświetlacz', 'wartość', '600zł', '.', 'dodatkowo', 'szkło', 'hartowane', '5d', 'oraz', 'pokrowiec', 'spigen', 'wartość', '80zl', '.', 'nie', 'sprzedaje', 'za', 'granice', '!', '!', '.', 'polecam']


In [3]:
tokenizer = BertTokenizer.from_pretrained('dkleczek/bert-base-polish-uncased-v1')
model = BertForMaskedLM.from_pretrained('dkleczek/bert-base-polish-uncased-v1')

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
encoded_corpus = tokenizer(text=preprocessed_data['Concatenated_description'].tolist(),
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=200,
                            return_attention_mask=True)

In [21]:
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']